# Convert Axis Marker .stl to Voxel Object

tomog data: ```Data/Raw Data/3D Objects/Axis Marker/Axis Marker.stl```
- object created in blender

Voxel Object: ```Data/Voxel Data/3D Objects/Axis Marker.pt```

- convert from stl to binary numpy mask
- Set spatial resolution = pitch
- Set background RI
- Set mask RI

In [ ]:
import os
import trimesh
import torch
import numpy as np

import matplotlib.pyplot as plt
import pyvista as pv
from matplotlib import colormaps

import scipy

In [ ]:
# Axis Marker Settings

blender_file = "../../Data/Raw Data/3D Objects/Axis Marker/Axis Marker.stl"

voxel_object_file = "../../Data/Voxel Data/3D Objects/Axis Marker.pt"

# probably also given in the .mat file 
unit = "um"
spatial_resolution = 0.1

object_RI = 1.33401
background_RI = 1.334

In [ ]:
# Load STL as mesh data
if not os.path.isfile(blender_file):
    raise Exception(f"File not found! \n {blender_file}")

mesh = trimesh.load_mesh(blender_file)

In [ ]:
# Convert Mesh into Voxel Mask (np.array)
voxel_mask = mesh.voxelized(pitch=spatial_resolution)
voxel_mask = voxel_mask.fill()          
voxel_mask = voxel_mask.matrix
voxel_mask = np.array(voxel_mask, dtype=np.float64)


In [ ]:
# Convert Mask to RI Values

ri_diff = object_RI - background_RI

ri_mask = voxel_mask * ri_diff + background_RI

In [ ]:
# check for miss alignment

plt.close("all")
data = ri_mask[:,:,100]
#data = np.roll(data,shift=1,axis=0)
#data = np.roll(data,shift=1,axis=1)
plt.imshow(data, cmap="jet")
plt.xticks(np.arange(0, data.shape[0], 25))
plt.yticks(np.arange(0, data.shape[0], 25))
plt.grid()
plt.show()

In [ ]:
# add some padding
voxel_object = np.pad(ri_mask, pad_width=(25,24), mode='constant', constant_values=background_RI)

print(voxel_object.shape)

In [ ]:
# Apply some blur
voxel_object = scipy.ndimage.gaussian_filter(voxel_object, sigma=2.0)

In [ ]:
# Check Symmetry

%matplotlib widget
plt.close("all")

fig, axs = plt.subplots(1, 3, figsize=(15,5))

xy_slice = voxel_object[:,:,124]
axs[0].imshow(xy_slice, cmap="jet")
axs[0].set_xticks(np.arange(0, xy_slice.shape[0], 5))
axs[0].set_yticks(np.arange(0, xy_slice.shape[0], 5))
axs[0].grid()

yz_slice = voxel_object[124,:,:]
axs[1].imshow(yz_slice, cmap="jet")
axs[1].set_xticks(np.arange(0, yz_slice.shape[0], 5))
axs[1].set_yticks(np.arange(0, yz_slice.shape[0], 5))
axs[1].grid()

xz_slice = voxel_object[:,124,:]
axs[2].imshow(xz_slice, cmap="jet")
axs[2].set_xticks(np.arange(0, xz_slice.shape[0], 5))
axs[2].set_yticks(np.arange(0, xz_slice.shape[0], 5))
axs[2].grid()
plt.show()

In [ ]:
colormap = "jet"
opacity = "sigmoid"

grid = pv.wrap(voxel_object)

cmap = colormaps[colormap]

# Create a volume plot
plotter = pv.Plotter(notebook=False)
plotter.add_volume(grid, cmap=cmap, opacity=opacity, n_colors=256)

plotter.show()
plotter.close()

In [ ]:
# Save Object as .pt file

voxel_object = torch.from_numpy(voxel_object)

spatial_res = torch.tensor([spatial_resolution, spatial_resolution, spatial_resolution]) #* unit

data_dict = {
    "data": voxel_object,
    "unit": "um",
    "spatial_resolution": spatial_res,
    "is_lefthand": False
    }

torch.save(data_dict, voxel_object_file)